In the 2 part we will manually craete transaction which spends from P2WKH to P2WSH.
Our P2WSH will be a 2-of-2 multisig as used for channel setup in LN. For this we will generate 2 private keys (here already generated, os.random) and create from them the address

In [62]:
# helper functions
import hashlib
import base58
import bech32
import ecdsa
from ecdsa import SigningKey, SECP256k1

def hash160(s):
    """Double hash using SHA256 and RIPEMD160."""
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def pubkey_to_bech32(pubkey, hrp='tb', witver=0):
    """Convert a compressed public key to a bech32 address."""
    # Convert hex pubkey to bytes
    pubkey_bytes = bytes.fromhex(pubkey)

    # Get the 20-byte pubkey hash (P2WPKH)
    pubkey_hash = hash160(pubkey_bytes)

    # Encode using bech32
    bech32_address = bech32.encode(hrp, witver, pubkey_hash)
    return bech32_address

def sha256(data):
    return hashlib.sha256(data).digest()


def create_pubkes(privKey_list):
    pubKey_list = []

    for privKey in privKey_list:
        sk = SigningKey.from_string(privKey, curve=SECP256k1)
        vk = sk.get_verifying_key()
        compressed_pubKey = vk.to_string("compressed")
        compressed_pubKey_hex = compressed_pubKey.hex()
        pubKey_list.append(compressed_pubKey_hex)


        print('compressed_pubKey_hex: ', compressed_pubKey_hex)
    
    return pubKey_list

def pubkey_to_P2WSH(pubkey, hrp='tb', witver=0):
    """Convert a compressed public key to a bech32 address."""
    # Convert hex pubkey to bytes
    pubkey_bytes = bytes.fromhex(pubkey)

    # Get the 20-byte pubkey hash (P2WPKH)
    pubkey_hash = sha256(pubkey_bytes)

    # Encode using bech32
    bech32_address = bech32.encode(hrp, witver, pubkey_hash)
    return bech32_address

def wif_to_hex(wif):
    # Decode WIF to bytes
    wif_bytes = base58.b58decode(wif)
    
    # Remove the version byte (0x80 for mainnet, 0xEF for testnet)
    # Remove the compression flag (0x01)
    # Remove the 4-byte checksum
    private_key_hex = wif_bytes[1:-5]
    
    return private_key_hex.hex()



In [63]:
# prev generated priv key with os.urandom(32) and derived pub key, addresses
# the pubkey and adresses can be derived with helper functions from the priv key
privKey1 = '06d7e4333783183b8438c42bc97cb9405ab3bf1f2f8292bdb4e2cb7d1b67a556'
privKey2 = 'ac4b7009c74ff52242e3ce54f3609a4045f362b57c2cb49c9d9dab307386b144'

pubKey1 = '03926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf88'
pubKey2 = '026b900b8a56753cd66b6b594a53c47f4c28dc77c579c49a76fd310857cf41d082'

bech32_address1 = 'tb1qn95j932qpsfgkseh8wf3c9lknrejfhr0ezkq0v'
bech32_address2 = 'tb1q2e0pcvvl3cxu97qss2rqhe6mxsrlwpvgzgt755'


## from adress
address_from = 'tb1qn95j932qpsfgkseh8wf3c9lknrejfhr0ezkq0v'
pubkey_from = '03926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf88'
privKey_WIF_from = 'cMp1CHaDsywtgNnB7q7bvq2iJGwQHDvUgSoobfgqBGGjtXwaTdJZ'



In [64]:
# lets create 2 of 2 multisig address

OP_m = '52'
OP_CHECKMULTISIG = 'ae'

pubKey1_len = hex(len(pubKey1)//2)[2:]
pubKey2_len = hex(len(pubKey2)//2)[2:]

raw_multiSig_Addr = OP_m + pubKey1_len + pubKey1 + pubKey2_len + pubKey2 + OP_m + OP_CHECKMULTISIG

print(raw_multiSig_Addr)

multiAddress = pubkey_to_P2WSH(raw_multiSig_Addr)

print(multiAddress)


522103926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf8821026b900b8a56753cd66b6b594a53c47f4c28dc77c579c49a76fd310857cf41d08252ae
tb1q4k7537w7x3uerrxa4jzgtpeaht0nspfeeegsmk60lr9crdaks32q2cf07u


In [65]:

# lets define the known variables
###  1  version
version = '02000000'

### 2 hashPrevouts
tx_hash = '38b40b2b7a41f78cbf6b8b7fb76fcc1f3a659e923a3a4ccea634d1c365cac273'
txid = bytes.fromhex(tx_hash)[::-1].hex()

vout = '01000000'

Prevouts = txid + vout


hashPrevouts = sha256(bytes.fromhex(sha256(bytes.fromhex(Prevouts)).hex())).hex()

print('hashPrevouts: ', hashPrevouts)

### 3 hashSequence
sequence = 'fdffffff'
hashSequence = sha256(bytes.fromhex(sha256(bytes.fromhex(sequence)).hex())).hex()

print('hashSequence: ', hashSequence)

### 4 outpoint
outpoint = txid + vout 

print('outpoint: ', outpoint)

### 5 scriptCode

# pubKey of the spender address
pubKey = bytes.fromhex(pubkey_from)
pubkey_sha256 = hashlib.sha256(pubKey).digest()
pubkey_hash = hashlib.new('ripemd160', pubkey_sha256).digest()


scriptCode = '1976a914' + pubkey_hash.hex() + '88ac'

print('scriptCode: ', scriptCode)


### 6 amount
# now lets convert 0.0047 btc that we wand to spend to satoshi
btc_value = 0.015
# Convert BTC to satoshi (1 BTC = 100,000,000 satoshis)
satoshis = int(btc_value * 100_000_000)
# Convert to 8-byte little endian format

amount = satoshis.to_bytes(8, 'little').hex().rjust(16, '0')
print(amount)

### 7 sequence
# use already defined 



### 8 hashOutputs
# scriptPubkey is OP_0 + len(hash256 of the raw_multiSig_Addr) + hash256 of the raw_multiSig_Addr

WitnessScript_hash = sha256(bytes.fromhex(raw_multiSig_Addr)).hex()
len_scriptPubkey = hex(len(WitnessScript_hash)//2)[2:]

scriptPubkey = '00' + len_scriptPubkey + WitnessScript_hash

print('scriptPubkey: ', scriptPubkey)




# im not clear if i need to add len of scriptPubkey to the hashOutputs or not
Outputs = amount +hex(len(scriptPubkey)//2)[2:] + scriptPubkey

print('outpus: ', Outputs)

hashOutputs = sha256(bytes.fromhex(sha256(bytes.fromhex(Outputs)).hex())).hex()


### 9 locktime
locktime = "00000000"

### 10  sig_hash
sig_hash = "01000000"

#  xxx
btc_in = 0.01522428

# Convert BTC to satoshi (1 BTC = 100,000,000 satoshis)
satoshis_in = int(btc_in * 100_000_000)
# Convert to 8-byte little endian format

inp_value = satoshis_in.to_bytes(8, 'little').hex().rjust(16, '0')




hashPrevouts:  2d91c0c1fa02c55edf2cfc6c3ca2cbd54a1fca09037a0d0ba227f1e4a8da05ca
hashSequence:  caf35e5224de16efa3ccaf41070f6e7b9432b6f79551e629fca9d1c03b43bc52
outpoint:  73c2ca65c3d134a6ce4c3a3a929e653a1fcc6fb77f8b6bbf8cf7417a2b0bb43801000000
scriptCode:  1976a914996922c5400c128b43373b931c17f698f324dc6f88ac
60e3160000000000
scriptPubkey:  0020adbd48f9de3479918cddac8485873dbadf380539ce510ddb4ff8cb81b7b68454
outpus:  60e3160000000000220020adbd48f9de3479918cddac8485873dbadf380539ce510ddb4ff8cb81b7b68454


In [66]:
##!!!##
# for signing the TX it is important to deffirentiate between value in outputhash and value in input. 
# value in output hash is what we want to send, value in input is what we have in the input UTXO

rawTx_toSign = (
    version         #1# 4 bytes: nVersion of the transaction
    + hashPrevouts  #2# 32 bytes: SHA256 hash of all previous outputs
    + hashSequence  #3# 32 bytes: SHA256 hash of input sequence numbers
    + outpoint      #4# (32-byte hash + 4-byte little endian) 
    + scriptCode    #5# Variable length: Script of the input (often P2PKH script for P2WPKH spends)
    + inp_value        #6# 8 bytes: Value of the UTXO being spent
    + sequence      #7# 4 bytes: nSequence of the input
    + hashOutputs   #8# 32 bytes: SHA256 hash of the outputs (if SIGHASH_ALL)
    + locktime      #9# 4 bytes: nLocktime of the transaction
    + sig_hash      #10# 4 bytes: Sighash type (e.g., SIGHASH_ALL)
)

In [67]:
# now we will sign the raw transaction with our private keys


privKey = wif_to_hex(privKey_WIF_from)

print('privKey: ', privKey)


rawTx_toSign_hash = sha256(bytes.fromhex(sha256(bytes.fromhex(rawTx_toSign)).hex())).hex()

from coincurve import PrivateKey

# signing key from private key deterministically

def signTx(rawTx_toSign_hash, privKey_hex):
    bytes_to_sign = bytes.fromhex(rawTx_toSign_hash)
    privkey = PrivateKey(bytes.fromhex(privKey_hex))
    signature = privkey.sign(bytes_to_sign, hasher=None)

    # also to signale that we used SigHash all we add 01 at the end
    signature += bytes.fromhex('01')

    scriptSig = signature.hex()
    Sig_len = hex(len(signature))[2:]
    scriptSig = Sig_len + scriptSig

    return scriptSig

scriptSig1 = signTx(rawTx_toSign_hash, privKey)

print(scriptSig1)


privKey:  06d7e4333783183b8438c42bc97cb9405ab3bf1f2f8292bdb4e2cb7d1b67a556
4730440220214a48f73b611de6913e8abbb1d61fe8786c6dddc0a1c2f5567dfa1c2b81a74a02205092b15d21a896e1a76b473190c53a9d077d0c2a74e9016e3cab9af68f8e9bc401


# Now we can start constructing the transaction. The structure of the transaction is very different from what has been signed (in legacy it was very similiar to what is signed, but here not)
rawTx = (
    version          #1# 4 bytes: Transaction version number
    + marker         #2# 1 byte: SegWit marker (0x00)
    + flag           #3# 1 byte: SegWit flag (0x01)
    + tx_in_count    #4# Variable length: Number of transaction inputs
    + txid           #5# 32 bytes: Transaction ID of the UTXO being spent
    + vout           #6# 4 bytes: Index of the UTXO in the transaction
    + scriptSig_empty      #7# Variable length: ScriptSig (often empty for SegWit)
    + sequence       #8# 4 bytes: Sequence number
    + tx_out_count   #9# Variable length: Number of transaction outputs
    + amount         #10# 8 bytes: Amount of satoshis to send to the recipient
    + scriptPubkey   #11# Variable length: Locking script (e.g., P2WPKH or P2WSH)
    # witness_field  Witness field, containing witness data for each input
    + witness_count
    + ScriptSig1
    + pubkey1
      
    + locktime       #13# 4 bytes: Time or block number when the transaction becomes valid
)



In [68]:
# lets define the missing variables
###  1  version (already defined)
print('version: ', version)

### 2 flag and marker (actually legacy tx expects after version, the number of inputs. 
# If it sees 00, it will know that it is SegWit with other tx structure - my understanding)


marker = '00'
flag = '01'

### 3 tx_in_count
tx_in_count = '01'

### 4 txid (already defined)
print('txid: ', txid)

### 5 vout (already defined)
print('vout: ', vout)

### 6 scriptSig (empty for segwit, will be placed in witness)
scriptSig_empty = '00'  

### 7 sequence (already defined)
print('sequence: ', sequence)

### 8 tx_out_count
tx_out_count = '01'

### 9 amount (already defined)
print('amount: ', amount)

### 10 scriptPubkey (already defined)

scriptPubkey = hex(len(scriptPubkey)//2)[2:] + scriptPubkey
print('scriptPubkey: ', scriptPubkey)

### 11 witness count (as i understand it is the number fields in witness...will be probably clear later)
witness_count = '02'

### 12 signature (already defined)
print('scriptSig1: ', scriptSig1)

### 13 pubkey (already defined)
pubkey1 = hex(len(pubkey_from)//2)[2:] + pubkey_from

print('pubkey: ', pubkey1)

### 14 locktime (already defined)
print('locktime: ', locktime)










version:  02000000
txid:  73c2ca65c3d134a6ce4c3a3a929e653a1fcc6fb77f8b6bbf8cf7417a2b0bb438
vout:  01000000
sequence:  fdffffff
amount:  60e3160000000000
scriptPubkey:  220020adbd48f9de3479918cddac8485873dbadf380539ce510ddb4ff8cb81b7b68454
scriptSig1:  4730440220214a48f73b611de6913e8abbb1d61fe8786c6dddc0a1c2f5567dfa1c2b81a74a02205092b15d21a896e1a76b473190c53a9d077d0c2a74e9016e3cab9af68f8e9bc401
pubkey:  2103926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf88
locktime:  00000000


In [69]:
rawTx = (
    version          #1# 4 bytes: Transaction version number
    + marker         #2# 1 byte: SegWit marker (0x00)
    + flag           #3# 1 byte: SegWit flag (0x01)
    + tx_in_count    #4# Variable length: Number of transaction inputs
    + txid           #5# 32 bytes: Transaction ID of the UTXO being spent
    + vout           #6# 4 bytes: Index of the UTXO in the transaction
    + scriptSig_empty      #7# Variable length: ScriptSig (often empty for SegWit)
    + sequence       #8# 4 bytes: Sequence number
    + tx_out_count   #9# Variable length: Number of transaction outputs
    + amount         #10# 8 bytes: Amount of satoshis to send to the recipient
    + scriptPubkey   #11# Variable length: Locking script (e.g., P2WPKH or P2WSH)
    # witness_field  Witness field, containing witness data for each input
    + witness_count
    + scriptSig1
    + pubkey1
      
    + locktime       #13# 4 bytes: Time or block number when the transaction becomes valid
)

print(rawTx)

0200000000010173c2ca65c3d134a6ce4c3a3a929e653a1fcc6fb77f8b6bbf8cf7417a2b0bb4380100000000fdffffff0160e3160000000000220020adbd48f9de3479918cddac8485873dbadf380539ce510ddb4ff8cb81b7b68454024730440220214a48f73b611de6913e8abbb1d61fe8786c6dddc0a1c2f5567dfa1c2b81a74a02205092b15d21a896e1a76b473190c53a9d077d0c2a74e9016e3cab9af68f8e9bc4012103926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf8800000000


This is the same output as produced by Bitcoin Core console:
0200000000010173c2ca65c3d134a6ce4c3a3a929e653a1fcc6fb77f8b6bbf8cf7417a2b0bb4380100000000fdffffff0160e3160000000000220020adbd48f9de3479918cddac8485873dbadf380539ce510ddb4ff8cb81b7b68454024730440220214a48f73b611de6913e8abbb1d61fe8786c6dddc0a1c2f5567dfa1c2b81a74a02205092b15d21a896e1a76b473190c53a9d077d0c2a74e9016e3cab9af68f8e9bc4012103926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf8800000000


In [70]:
rawTx_console = '0200000000010173c2ca65c3d134a6ce4c3a3a929e653a1fcc6fb77f8b6bbf8cf7417a2b0bb4380100000000fdffffff0160e3160000000000220020adbd48f9de3479918cddac8485873dbadf380539ce510ddb4ff8cb81b7b68454024730440220214a48f73b611de6913e8abbb1d61fe8786c6dddc0a1c2f5567dfa1c2b81a74a02205092b15d21a896e1a76b473190c53a9d077d0c2a74e9016e3cab9af68f8e9bc4012103926477faac658a56cbb36bc565a4f58743c2cafaa12926d37fb0f23eb409bf8800000000'

print(rawTx_console==rawTx)

True
